In [ ]:
import pygame
import random
# Initialize Pygame
pygame.init()
# Screen dimensions
screen_width = 450  # Increased width to accommodate next piece preview and score
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))


# Colors
colors = [
    (0, 0, 0),      # Black
    (255, 0, 0),    # Red
    (0, 255, 0),    # Green
    (0, 0, 255),    # Blue
    (255, 255, 0),  # Yellow
    (255, 165, 0),  # Orange
    (0, 255, 255),  # Cyan
    (128, 0, 128)   # Purple
]
# Tetromino shapes
tetrominoes = [
    [[1, 1, 1, 1]],
    [[1, 1], [1, 1]],
    [[1, 1, 0], [0, 1, 1]],
    [[0, 1, 1], [1, 1, 0]],
    [[1, 0, 0], [1, 1, 1]],
    [[0, 0, 1], [1, 1, 1]],
    [[0, 1, 0], [1, 1, 1]]
]


# Game variables
grid = [[0 for _ in range(10)] for _ in range(20)]
current_piece = random.choice(tetrominoes)
current_color = random.randint(1, len(colors) - 1)
next_piece = random.choice(tetrominoes)
next_color = random.randint(1, len(colors) - 1)
current_pos = [0, 4]
clock = pygame.time.Clock()
fall_time = 0
score = 0
level = 1
# Font for displaying score
font = pygame.font.SysFont('Arial', 24)


def draw_grid(surface):
    for y in range(20):
        for x in range(10):
            pygame.draw.rect(surface, colors[grid[y][x]], (x * 30, y * 30, 30, 30), 0)
            pygame.draw.rect(surface, (128, 128, 128), (x * 30, y * 30, 30, 30), 1)


def draw_piece(surface, piece, pos, color):
    for y, row in enumerate(piece):
        for x, cell in enumerate(row):
            if cell:
                pygame.draw.rect(surface, colors[color], ((pos[1] + x) * 30, (pos[0] + y) * 30, 30, 30), 0)
                pygame.draw.rect(surface, (128, 128, 128), ((pos[1] + x) * 30, (pos[0] + y) * 30, 30, 30), 1)


def draw_next_piece(surface, piece, color):
    for y, row in enumerate(piece):
        for x, cell in enumerate(row):
            if cell:
                pygame.draw.rect(surface, colors[color], (320 + x * 30, 100 + y * 30, 30, 30), 0)
                pygame.draw.rect(surface, (128, 128, 128), (320 + x * 30, 100 + y * 30, 30, 30), 1)


def draw_score(surface, score):
    score_text = font.render(f'Score: {score}', True, (255, 255, 255))
    surface.blit(score_text, (320, 50))


def check_collision(piece, pos):
    for y, row in enumerate(piece):
        for x, cell in enumerate(row):
            if cell:
                if y + pos[0] >= 20 or x + pos[1] < 0 or x + pos[1] >= 10 or grid[y + pos[0]][x + pos[1]]:
                    return True
    return False


def lock_piece(piece, pos, color):
    global grid, current_piece, current_color, next_piece, next_color, current_pos, score, level
    for y, row in enumerate(piece):
        for x, cell in enumerate(row):
            if cell:
                grid[y + pos[0]][x + pos[1]] = color
    current_piece = next_piece
    current_color = next_color
    next_piece = random.choice(tetrominoes)
    next_color = random.randint(1, len(colors) - 1)
    current_pos = [0, 4]
    clear_lines()
    if check_collision(current_piece, current_pos):
        grid = [[0 for _ in range(10)] for _ in range(20)]
        score = 0
        level = 1


def clear_lines():
    global grid, score, level
    lines_cleared = 0
    for y in range(20):
        if all(grid[y]):
            del grid[y]
            grid.insert(0, [0 for _ in range(10)])
            lines_cleared += 1
    score += lines_cleared * 100
    if score // 1000 > level:
        level = score // 1000 + 1


running = True
while running:
    screen.fill((0, 0, 0))
    draw_grid(screen)
    draw_piece(screen, current_piece, current_pos, current_color)
    draw_next_piece(screen, next_piece, next_color)
    draw_score(screen, score)
    pygame.display.flip()
    fall_speed = 300 - (level - 1) * 20
    fall_time += clock.get_rawtime()
    clock.tick()
    if fall_time > fall_speed:
        fall_time = 0
        current_pos[0] += 1
        if check_collision(current_piece, current_pos):
            current_pos[0] -= 1
            lock_piece(current_piece, current_pos, current_color)


    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                current_pos[1] -= 1
                if check_collision(current_piece, current_pos):
                    current_pos[1] += 1
            elif event.key == pygame.K_RIGHT:
                current_pos[1] += 1
                if check_collision(current_piece, current_pos):
                    current_pos[1] -= 1
            elif event.key == pygame.K_DOWN:
                current_pos[0] += 1
                if check_collision(current_piece, current_pos):
                    current_pos[0] -= 1
                    lock_piece(current_piece, current_pos, current_color)
            elif event.key == pygame.K_UP:
                rotated_piece = [list(row) for row in zip(*current_piece[::-1])]
                if not check_collision(rotated_piece, current_pos):
                    current_piece = rotated_piece


pygame.quit()